In [17]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [18]:
test_csv = pd.read_csv("ruw_data/test.csv")
submission_csv = pd.read_csv("prediction/submission.csv")

## 1. 정답과 모델의 예측값 비교하기

In [19]:
sentences = test_csv["sentence"]
subject_entities, object_entities = test_csv["subject_entity"], test_csv["object_entity"] 

my_probs = submission_csv["probs"]
my_labels = submission_csv["pred_label"]
ans_labels = test_csv["label"]

In [4]:
i = -1
cnt = 0

while i < len(my_labels)-1:
    i += 1
    if my_labels[i] == ans_labels[i]:
        continue
    cnt += 1
    print(f"{cnt}: {sentences[i]}")
    print(f"sub:{eval(subject_entities[i])['word']}, obj:{eval(object_entities[i])['word']}")
    print(f"예측: {my_labels[i]}, sota: {ans_labels[i]}")
    print()
print(f"num of diff: {cnt} / {i}")

1: 즉 푸틴 대통령 정부가 내세운 정책에 대해서 야당과 러시아 국민들의 반대로 인하여 정권 자체가 휘청거리고 있다.
sub:푸틴 대통령 정부, obj:러시아 국민들
예측: no_relation, sota: org:top_members/employees

2: 이에 형식상으로는 소련에서 가장 큰 공화국이 탈퇴한 것으로 보였으나 러시아 측에서는 '더 이상 존재하지 않는 나라'에서 탈퇴하는 것은 불가능하므로 이 같은 말은 사실이 아니라고 말했다.
sub:러시아, obj:소련
예측: no_relation, sota: org:member_of

3: 러시아는 군사력은 세계에서 2번째라고 불리우며 국방비 지출도 4번째로 많다.
sub:러시아, obj:국방비
예측: no_relation, sota: org:property

4: EU 외무장관들은 탈퇴 지역의 승인에 찬성표를 던진 모든 두마 회원들을 블랙리스트에 올렸고, EU 투자자들의 러시아 국채 거래를 금지했으며, 분리주의 단체들과의 수출입도 목표물로 삼았다.
sub:분리주의 단체들, obj:EU 투자자들
예측: org:friendly, sota: no_relation

5: 그러나 시브산카르는 인도 내각의 속하는 것이 아니어서 인도 정부의 공식 입장으로 보이기는 어렵다.
sub:인도, obj:시브산카르
예측: org:top_members/employees, sota: no_relation

6: 우크라이나 내무부는 우크라이나군이 4월 23일 슬로비얀스크 근처의 시비아토고르스크(Sviatogorsk) 도시를 탈환했다고 말했다.
sub:우크라이나군, obj:우크라이나 내무부
예측: org:member_of, sota: no_relation

7: 슬라브족들의 역사는 대체적으로 게르만족 같은 잦은 외세에 시달린 것을 이겨내지 못하여 이민족의 지배에 점철된 역사이며, 슬라브족들은 오래 게르만 등 이민족의 지배 하에서 민족적, 문화적 국가나 연합체가 나타나지 못한 채 게르만족들의 영토에서 농노화되어 있었다.
sub:슬라브족,

## 2.no_relation 제외 f1 score 계산

In [49]:
import numpy as np
from sklearn.metrics import f1_score, precision_recall_curve, auc

def ruw_re_micro_f1(preds, labels):
    """Russia-Ukraine War RE micro f1 (except no_relation)"""
    label_list = ['no_relation', 'per:title', 'org:member_of', 
        'org:alternate_names', 'org:top_members/employees', 'org:hostile', 
        'org:friendly', 'org:property', 'eve:place', 'eve:date'] 

    # return f1_score(labels, preds, average="micro", labels=label_list[1:]) * 100.0
    micro_f1 = f1_score(labels, preds, average="micro", labels=label_list[1:]) * 100.0
    macro_f1 = f1_score(labels, preds, average="macro", labels=label_list[1:]) * 100.0
    weighted_f1 = f1_score(labels, preds, average="weighted", labels=label_list[1:]) * 100.0
    average_f1 = f1_score(ans_labels, my_labels, average=None) * 100.0
    return micro_f1, macro_f1, weighted_f1, average_f1



def ruw_re_auprc(probs, labels):
    """Russia-Ukraine War RE AUPRC (with no_relation)"""
    labels = np.eye(10)[labels]

    score = np.zeros((10,))
    for c in range(10):
        targets_c = labels.take([c], axis=1).ravel()
        preds_c = probs.take([c], axis=1).ravel()
        precision, recall, _ = precision_recall_curve(targets_c, preds_c)
        score[c] = auc(recall, precision)
    return np.average(score) * 100.0

In [50]:
test_csv = pd.read_csv("ruw_data/test.csv")
submission_csv = pd.read_csv("prediction/submission.csv")
print('전체 test data:', len(test_csv))
print('전체 sub data:', len(submission_csv))


my_labels = submission_csv["pred_label"].to_list()
ans_labels = test_csv["label"].to_list()
micro_f1, macro_f1, weighted_f1, average_f1 = ruw_re_micro_f1(my_labels, ans_labels)

print('micro f1 score:', micro_f1)
print('macro f1 score:', macro_f1)
print('weighted f1 score:', weighted_f1)
print('f1 score of each class:', average_f1)

전체 test data: 177
전체 sub data: 177
micro f1 score: 88.15165876777252
macro f1 score: 81.49452683935442
weighted f1 score: 87.74281943767956
f1 score of each class: [ 97.2972973  100.          82.51748252  92.30769231  33.33333333
  75.86206897  85.          76.92307692  72.72727273 100.        ]
